In [14]:
import os
from datetime import timedelta
from couchbase.exceptions import CouchbaseException
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions
from twarc import Twarc2
import pandas as pd
import json

In [2]:
# Configuración de Couchbase
endpoint = "couchbases://cb.pps-dendznwwlsxh.cloud.couchbase.com"
username = "BDD_PULSOSPOLITICOS"
password = "Administrador32#"  # Reemplazar con la contraseña real
bucket_name = "BDD_PULSOSPOLITICOS"
scope_name = "OpinionPublicaPolitica"
collection_name = "OpinionesPolitica"

In [4]:
# Load Twitter API keys from .env file
twitter_consumer_key = os.getenv("TWITTER_CONSUMER_KEY")
twitter_consumer_secret = os.getenv("TWITTER_CONSUMER_SECRET")
twitter_access_token = os.getenv("TWITTER_ACCESS_TOKEN")
twitter_access_token_secret = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

In [ ]:
# Initialize Twitter API client
twarc = Twarc2(consumer_key=twitter_consumer_key,
               consumer_secret=twitter_consumer_secret,
               access_token=twitter_access_token,
               access_token_secret=twitter_access_token_secret)


In [6]:
# Couchbase connection setup
auth = PasswordAuthenticator(username, password)
options = ClusterOptions(auth)
options.apply_profile("wan_development")

cluster = Cluster(endpoint, options)
cluster.wait_until_ready(timedelta(seconds=5))
cb = cluster.bucket(bucket_name)
cb_coll = cb.scope(scope_name).collection(collection_name)

In [7]:
# Twitter query setup
twitter_query = "#EcuadorPolitics lang:es"
tweets_data = []


In [ ]:
# Simulación de tweets obtenidos (para demostración)
fetched_tweets = [
    {"id": "1", "text": "Tweet de ejemplo 1", "author_id": "user1", "created_at": "2025-01-26T12:00:00Z"},
    {"id": "2", "text": "Tweet de ejemplo 2", "author_id": "user2", "created_at": "2025-01-26T12:05:00Z"}
]
tweets_data = []

In [ ]:
for tweet in fetched_tweets:
    tweet_data = {
        "id": tweet["id"],
        "text": tweet["text"],
        "author": tweet["author_id"],
        "created_at": tweet["created_at"]
    }
    tweets_data.append(tweet_data)


    try:
        cb_coll.insert(tweet_data["id"], tweet_data)
        print(f"Tweet {tweet_data['id']} inserted into Couchbase.")
    except CouchbaseException as e:
        print(f"Error inserting tweet {tweet_data['id']}: {e}")

In [18]:
# Cargar datos desde el archivo JSON
try:
    with open("tweets_respuestas.json", "r", encoding="utf-8") as f:
        tweets_data = json.load(f)
    print("JSON file loaded successfully.")
except Exception as e:
    print(f"Error loading JSON file: {e}")

JSON file loaded successfully.


In [19]:
# Transformar JSON a CSV y guardar en un archivo CSV
try:
    df = pd.DataFrame(tweets_data)
    df.to_csv('tweets_respuestas.csv', index=False, encoding='utf-8')
    print("CSV file created successfully.")
except Exception as e:
    print(f"Error creating CSV file: {e}")

CSV file created successfully.


In [ ]:
# Subir el archivo CSV a Couchbase 
try:
    with open('tweets_respuestas.csv', 'r', encoding='utf-8') as f:
        csv_content = f.read()
    
    cb_coll.insert('tweets_respuestas_csv', {'content': csv_content})
    print("CSV content inserted into Couchbase.")
except CouchbaseException as e:
    print(f"Error inserting CSV content into Couchbase: {e}")
except Exception as e:
    print(f"General error: {e}")

CSV content inserted into Couchbase.
